<a href="https://colab.research.google.com/github/hygjob/keras_exercise/blob/master/intro_functional_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Getting started with the Keras functional API

In [0]:
# First example: a densely-connected network

In [1]:
from keras.layers import Input, Dense
from keras.models import Model

# This returns a tensor
inputs = Input(shape=(784,))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(data, labels)  # starts training

Using TensorFlow backend.
W0830 02:13:39.527831 140033199413120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0830 02:13:39.565962 140033199413120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0830 02:13:39.576590 140033199413120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0830 02:13:39.624075 140033199413120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0830 02:13:39.651505 14003319

NameError: ignored

In [0]:
# All models are callable, just like layers

x = Input(shape=(784,))
# This works, and returns the 10-way softmax we defined above.
y = model(x)

In [5]:
print(x)
print(y)

Tensor("input_2:0", shape=(?, 784), dtype=float32)
Tensor("model_1/dense_3/Softmax:0", shape=(?, 10), dtype=float32)


In [0]:
# This can allow, for instance, to quickly create models that can process sequences of inputs. 
# You could turn an image classification model into a video classification model, in just one line.

In [0]:
from keras.layers import TimeDistributed

# Input tensor for sequences of 20 timesteps,
# each containing a 784-dimensional vector
input_sequences = Input(shape=(20, 784))

# This applies our previous model to every timestep in the input sequences.
# the output of the previous model was a 10-way softmax,
# so the output of the layer below will be a sequence of 20 vectors of size 10.
processed_sequences = TimeDistributed(model)(input_sequences)

In [7]:
print(input_sequences.shape)

(?, 20, 784)


In [8]:
print(processed_sequences)

Tensor("time_distributed_1/Reshape_1:0", shape=(?, 20, 10), dtype=float32)


In [0]:
 # Multi-input and multi-output models

![함수형 API](https://s3.amazonaws.com/keras.io/img/multi-input-multi-output-graph.png)

The main input will receive the headline, as a sequence of integers (each integer encodes a word). The integers will be between 1 and 10,000 (a vocabulary of 10,000 words) and the sequences will be 100 words long.

In [0]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

# Headline input: meant to receive sequences of 100 integers, between 1 and 10000.
# Note that we can name any layer by passing it a "name" argument.
main_input = Input(shape=(100,), dtype='int32', name='main_input')

# This embedding layer will encode the input sequence
# into a sequence of dense 512-dimensional vectors.
x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)

# A LSTM will transform the vector sequence into a single vector,
# containing information about the entire sequence
lstm_out = LSTM(32)(x)

In [0]:
auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)

In [0]:
LSTM 출력값과 추가 입력값을 결합해 보자.

In [0]:
import keras

auxiliary_input = Input(shape=(5,), name='aux_input')
x = keras.layers.concatenate([lstm_out, auxiliary_input])

# We stack a deep densely-connected network on top
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# And finally we add the main logistic regression layer
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

In [0]:
2개의 입력과 2개의 출력을 정의해 보자.

In [0]:
model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output, auxiliary_output])

In [0]:
추가 값에 0.2의 가중치를 적용해 보자. 

In [14]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy',
              loss_weights=[1., 0.2])

W0830 02:31:17.039524 140033199413120 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
입력과 출력을 배열형식으로 넣고 학습을 시켜보자

In [15]:
model.fit([headline_data, additional_data], [labels, labels],
          epochs=50, batch_size=32)

NameError: ignored

In [0]:
입력시 이름을 주어 값을 구별할 수도 있다.

In [16]:
model.compile(optimizer='rmsprop',
              loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
              loss_weights={'main_output': 1., 'aux_output': 0.2})

# And trained it via:
model.fit({'main_input': headline_data, 'aux_input': additional_data},
          {'main_output': labels, 'aux_output': labels},
          epochs=50, batch_size=32)

NameError: ignored